In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
# to change api, go to https://console.cloud.google.com/welcome?project=accessing-files
# Do not push client_secrets.json to github. It's in the .gitignore file, and can be changed or downloaded from google console. 


In [4]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=192179164139-4ta4iq8hm9damrd6bqmo3dk57spgmhh5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&access_type=offline&response_type=code

Authentication successful.


In [16]:
# upload file to shared google drive folder
def upload_to_google_drive(drive, filename, shared_drive_foldername):
    shared_drive_folder = drive.ListFile({'q': "title='" + shared_drive_foldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]
    shared_drive_folderid = shared_drive_folder['id']
    # if file already exists, delete it
    file_list = drive.ListFile({'q': "title='" + filename + "' and trashed=false"}).GetList()
    if len(file_list) > 0:
        file_list[0].Trash()
    file = drive.CreateFile({'title': filename, 'parents': [{'id': shared_drive_folderid}]})
    file.SetContentFile('tmp_files/' + filename)
    file.Upload()

filename = 'emily.stl'
shared_drive_foldername = 'collab-intell-shared'

upload_to_google_drive(drive, filename, shared_drive_foldername)

In [20]:
# upload file to subfolder of shared google drive folder
def upload_to_google_drive_subfolder(drive, filename, shared_drive_foldername, subfoldername):
    shared_drive_folder = drive.ListFile({'q': "title='" + shared_drive_foldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]
    shared_drive_folderid = shared_drive_folder['id']
    # create subfolder if it doesn't exist
    if len(drive.ListFile({'q': "title='" + subfoldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()) == 0:
        subfolder = drive.CreateFile({'title': subfoldername, 'parents': [{'id': shared_drive_folderid}], 'mimeType': 'application/vnd.google-apps.folder'})
        subfolder.Upload()
    else:
        subfolder = drive.ListFile({'q': "title='" + subfoldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]
    subfolderid = subfolder['id']
    # if file already exists in the subfolder, delete it
    file_list = drive.ListFile({'q': "title='" + filename + "' and trashed=false and '" + subfolderid + "' in parents"}).GetList()
    if len(file_list) > 0:
        file_list[0].Trash()
    
    file = drive.CreateFile({'title': filename, 'parents': [{'id': subfolderid}]})
    file.SetContentFile('tmp_files/' + filename)
    file.Upload()

filename = 'emily.stl'
shared_drive_foldername = 'collab-intell-shared'
subfoldername = 'emily'

upload_to_google_drive_subfolder(drive, filename, shared_drive_foldername, subfoldername)

In [33]:
# upload folder to subfolder of shared google drive folder
def upload_folder_to_google_drive_subfolder(drive, foldername, shared_drive_foldername, subfoldername):
    shared_drive_folder = drive.ListFile({'q': "title='" + shared_drive_foldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]
    shared_drive_folderid = shared_drive_folder['id']
    # create subfolder if it doesn't exist
    if len(drive.ListFile({'q': "title='" + subfoldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()) == 0:
        subfolder = drive.CreateFile({'title': subfoldername, 'parents': [{'id': shared_drive_folderid}], 'mimeType': 'application/vnd.google-apps.folder'})
        subfolder.Upload()
    else:
        subfolder = drive.ListFile({'q': "title='" + subfoldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]
    subfolderid = subfolder['id']
    # create folder if it doesn't exist
    destination_foldername = foldername.split('/')[-1]
    if len(drive.ListFile({'q': "title='" + destination_foldername + "' and trashed=false and '" + subfolderid + "' in parents and mimeType='application/vnd.google-apps.folder'"}).GetList()) == 0:
        folder = drive.CreateFile({'title': destination_foldername, 'parents': [{'id': subfolderid}], 'mimeType': 'application/vnd.google-apps.folder'})
        folder.Upload()
    else:
        folder = drive.ListFile({'q': "title='" + destination_foldername + "' and trashed=false and '" + subfolderid + "' in parents and mimeType='application/vnd.google-apps.folder'"}).GetList()[0]
    folderid = folder['id']
    for file in os.listdir(foldername):
        # check if it's a folder
        if os.path.isdir(os.path.join(foldername, file)):
            print('Folder ' + file + ' not uploaded because it is a folder')
        else:
            file_drive = drive.CreateFile({'title': file, 'parents': [{'id': folderid}]})
            file_drive.SetContentFile(foldername + '/' + file)
            file_drive.Upload()


foldername = 'tmp_files/sim0_run0'
shared_drive_foldername = 'collab-intell-shared'
subfoldername = 'emily'

upload_folder_to_google_drive_subfolder(drive, foldername, shared_drive_foldername, subfoldername)

In [34]:
# upload folder to shared google drive folder
def upload_folder_to_google_drive(drive, foldername, shared_drive_foldername):
    shared_drive_folder = drive.ListFile({'q': "title='" + shared_drive_foldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]
    shared_drive_folderid = shared_drive_folder['id']
    # create folder if it doesn't exist
    destination_foldername = foldername.split('/')[-1]
    if len(drive.ListFile({'q': "title='" + destination_foldername + "' and trashed=false and '" + shared_drive_folderid + "' in parents and mimeType='application/vnd.google-apps.folder'"}).GetList()) == 0:
        folder = drive.CreateFile({'title': destination_foldername, 'parents': [{'id': shared_drive_folderid}], 'mimeType': 'application/vnd.google-apps.folder'})
        folder.Upload()
    else:
        folder = drive.ListFile({'q': "title='" + destination_foldername + "' and trashed=false and '" + shared_drive_folderid + "' in parents and mimeType='application/vnd.google-apps.folder'"}).GetList()[0]
    folderid = folder['id']
    for file in os.listdir(foldername):
        file_drive = drive.CreateFile({'title': file, 'parents': [{'id': folderid}]})
        file_drive.SetContentFile(foldername + '/' + file)
        file_drive.Upload()

foldername = 'tmp_files/sim0_run0'
shared_drive_foldername = 'collab-intell-shared'

upload_folder_to_google_drive(drive, foldername, shared_drive_foldername)

In [24]:
# list all files in shared google drive folder
shared_drive_foldername = 'collab-intell-shared'

shared_drive_folder = drive.ListFile({'q': "title='" + shared_drive_foldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]
shared_drive_folderid = shared_drive_folder['id']
file_list = drive.ListFile({'q': "'" + shared_drive_folderid + "' in parents and trashed=false"}).GetList()
for file in file_list:
    print('title: %s, id: %s' % (file['title'], file['id']))
    

title: rewardlocsDF.csv, id: 1b6PokIFl_v_dJvnr_KhLjjuoHV7wI28e
title: birdlocsDF.csv, id: 1oOmfZP9f8p7ycZJMsgLzDROm2uxEuQfy


In [25]:
# download file from shared google drive folder
def download_from_google_drive(drive, filename, shared_drive_foldername):
    shared_drive_folder = drive.ListFile({'q': "title='" + shared_drive_foldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]
    shared_drive_folderid = shared_drive_folder['id']
    file = drive.ListFile({'q': "title='" + filename + "' and '" + shared_drive_folderid + "' in parents and trashed=false"}).GetList()[0]
    file.GetContentFile(os.path.join('tmp_files/', filename))

shared_drive_foldername = 'collab-intell-shared'
filename = 'emily.stl'
download_from_google_drive(drive, filename, shared_drive_foldername)


In [30]:
# download file from subfolder of shared google drive folder
def download_from_google_drive_subfolder(drive, filename, shared_drive_foldername, subfoldername):
    shared_drive_folder = drive.ListFile({'q': "title='" + shared_drive_foldername + "' and mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()[0]
    shared_drive_folderid = shared_drive_folder['id']
    subfolder = drive.ListFile({'q': "title='" + subfoldername + "' and '" + shared_drive_folderid + "' in parents and trashed=false"}).GetList()[0]
    subfolderid = subfolder['id']
    file = drive.ListFile({'q': "title='" + filename + "' and '" + subfolderid + "' in parents and trashed=false"}).GetList()[0]
    file.GetContentFile(os.path.join('tmp_files/', filename))


shared_drive_foldername = 'collab-intell-shared'
subfolder_name = 'emily'
filename = 'rewardlocsDF.csv'

download_from_google_drive_subfolder(drive, filename, shared_drive_foldername, subfolder_name)
